In [1]:
#Import the relevand section of the NLP library
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

import re

from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/Alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
seinfeld_directory = 'Seinfeld_Episodes/Season_5/'

seinfeld_season_5_episodes = ['S05_E01_The_Mango.txt', 'S05_E02_The_Puffy_Shirt.txt',
                              'S05_E03_The_Glasses.txt', 'S05_E04_The_Sniffing_Accountant.txt',
                              'S05_E05_The_Bris.txt', 'S05_E06_The_Lip_Reader.txt',
                              'S05_E07_The_Non_Fat_Yogurt.txt', 'S05_E08_The_Barber.txt',
                              'S05_E09_The_Masseuse.txt', 'S05_E10_The_Cigar_Store_Indian.txt',
                              'S05_E11_The_Conversion.txt', 'S05_E12_The_Stall.txt',
                              'S05_E13_The_Dinner_Party.txt', 'S05_E14_The_Marine_Biologist.txt',
                              'S05_E15_The_Pie.txt', 'S05_E16_The_Stand-In.txt',
                              'S05_E17_The_Wife.txt', 'S05_E18_The_Raincoats_Part_1.txt',
                              'S05_E19_The_Raincoats_Part_2.txt', 'S05_E20_The_Fire.txt',
                              'S05_E21_The_Hamptons.txt', 'S05_E22_The Opposite.txt']

In [3]:
with open(seinfeld_directory + seinfeld_season_5_episodes[1], 'r') as file:
    raw_text_episode_2 = file.read().replace('\n', ' ')
    #.replace('[','(').replace(']',')')
raw_text_episode_2[:1050]

"[Setting: Jerry's apartment] (Jerry and George are waiting for Kramer, so he can help them move George's stuff back into his parent's house) GEORGE: I can't believe this! JERRY: Oh, it won't be for that long. GEORGE: How can I do this?! How can I move back in with those people? Please, tell me! They're insane! You know that. JERRY: Hey, my parents are just as crazy as your parents. GEORGE: How can you compare you parents to my parents?! JERRY: My father has never thrown anything out. Ever! GEORGE: My father wears his sneakers in the pool! Sneakers! JERRY: My mother has never set foot in a natural body of water. GEORGE: (Showing Jerry up) Listen carefully. My mother has never laughed. Ever. Not a giggle, not a chuckle, not a tee-hee.. never went 'Ha!' JERRY: A smirk? GEORGE: Maybe!.. And I'm moving back in there! JERRY: I told you I'd lend you the money for the rent. GEORGE: No, no, no, no. Borrowing money from a friend is like having sex. It just completely changes the relationship. (

In [33]:
def cleaned_episode(raw_text):
    cleaned_episode = []
    
    raw_text_no_notes = re.sub("[\(\[].*?[\)\]]", "", raw_text)
    
    symbols =  ",.?!''\\"
    cleaned_text = raw_text_no_notes.replace(symbols, '').lower()
    cleaned_text.replace("\'", "")
    return cleaned_text


In [34]:
cleaned_episode_2 = cleaned_episode(raw_text_episode_2)

In [35]:
cleaned_episode_2

'  george: i can\'t believe this! jerry: oh, it won\'t be for that long. george: how can i do this?! how can i move back in with those people? please, tell me! they\'re insane! you know that. jerry: hey, my parents are just as crazy as your parents. george: how can you compare you parents to my parents?! jerry: my father has never thrown anything out. ever! george: my father wears his sneakers in the pool! sneakers! jerry: my mother has never set foot in a natural body of water. george:  listen carefully. my mother has never laughed. ever. not a giggle, not a chuckle, not a tee-hee.. never went \'ha!\' jerry: a smirk? george: maybe!.. and i\'m moving back in there! jerry: i told you i\'d lend you the money for the rent. george: no, no, no, no. borrowing money from a friend is like having sex. it just completely changes the relationship.  kramer: alright. i\'m ready.  you know, i still don\'t understand - why do you want to move back in with your parents? george: i don\'t want to! i\'m 

# Part Two: tokenize/stemming/lemmatization

In [40]:
def tokenize(cleaned_text):
    return word_tokenize(cleaned_text)

In [41]:
tokenize(cleaned_episode_2)

['george',
 ':',
 'i',
 'ca',
 "n't",
 'believe',
 'this',
 '!',
 'jerry',
 ':',
 'oh',
 ',',
 'it',
 'wo',
 "n't",
 'be',
 'for',
 'that',
 'long',
 '.',
 'george',
 ':',
 'how',
 'can',
 'i',
 'do',
 'this',
 '?',
 '!',
 'how',
 'can',
 'i',
 'move',
 'back',
 'in',
 'with',
 'those',
 'people',
 '?',
 'please',
 ',',
 'tell',
 'me',
 '!',
 'they',
 "'re",
 'insane',
 '!',
 'you',
 'know',
 'that',
 '.',
 'jerry',
 ':',
 'hey',
 ',',
 'my',
 'parents',
 'are',
 'just',
 'as',
 'crazy',
 'as',
 'your',
 'parents',
 '.',
 'george',
 ':',
 'how',
 'can',
 'you',
 'compare',
 'you',
 'parents',
 'to',
 'my',
 'parents',
 '?',
 '!',
 'jerry',
 ':',
 'my',
 'father',
 'has',
 'never',
 'thrown',
 'anything',
 'out',
 '.',
 'ever',
 '!',
 'george',
 ':',
 'my',
 'father',
 'wears',
 'his',
 'sneakers',
 'in',
 'the',
 'pool',
 '!',
 'sneakers',
 '!',
 'jerry',
 ':',
 'my',
 'mother',
 'has',
 'never',
 'set',
 'foot',
 'in',
 'a',
 'natural',
 'body',
 'of',
 'water',
 '.',
 'george',
 ':',